# Assignment 2: Long Short-Term Memory

*Author:* Thomas Adler

*Copyright statement:* This  material,  no  matter  whether  in  printed  or  electronic  form,  may  be  used  for  personal  and non-commercial educational use only.  Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed or in electronic form, requires explicit prior acceptance of the authors.

## Exercise 1: Analysis of the Jordan network

In the previous assignment we found that the Jordan network is incapable of memorizing information over longer time spans. Now we want to delve deeper into why this is case and how to solve this problem. To learn long-term dependencies, it is necessary that the gradient carries the error signal backwards in time. For instance, if the output at time $t$ depends on the input at time $s$ with $s < t$, then the Jacobian 
$$
J(t, s) = \frac{\partial a(t)}{\partial a(s)}
$$
is responsible for carrying the error signals from time $t$ backwards to time $s$. Calculate this Jacobian for the Jordan network and elaborate on the numerical stability of $J(t, s)$ for long time spans, i.e., when $t-s$ becomes large. Why is the Jordan network incapable of learning long-term dependencies?

########## YOUR SOLUTION HERE ##########

$$
J(t,s) = \frac{\partial a(t)}{\partial a(s)}, s<t
$$

$$
\frac{\partial a(t)}{\partial a(s)} = \frac{\partial a(t)}{\partial s(t)}\frac{\partial s(t)}{\partial \hat y(t-1)}\frac{\partial \hat{y}(t-1)}{\partial z(t-1)}\frac{\partial z(t-1)}{\partial a(t-1)}\frac{\partial a(t-1)}{\partial a(s)}
$$
applying the above extension recursively:
$$
\frac{\partial a(t)}{\partial a(s)} = \prod_{i=1}^{t-s} \frac{\partial a((t+1)-i)}{\partial s((t+1)-i)}\frac{\partial s((t+1)-i)}{\partial \hat y(t-i)}\frac{\partial \hat{y}(t-i)}{\partial z(t-i)}\frac{\partial z(t-i)}{\partial a(t-i)}
$$
the Jacobian Matrix $J(t,s)$ is then:
$$
\frac{\partial a(t)}{\partial a(s)} = \prod_{i=1}^{t-s} (1-tanh^2(s(t))) \, R \, \sigma '(z) \, V
$$

Which scales exponentially with $t-s$. This that the Jacobian is numerically unstable  when $t-s$ becomes large. The Jordan network is incabable of learning long-term dependencies because the error either explodes or vanishes as dependencies grow further apart in time.

########## YOUR SOLUTION HERE ##########


The following figure depicts the LSTM architecture (without forget gate, which was introduced later). 

<img src="lstm_noFG.png" alt="LSTM" width="600"/>

We have
$$
a(t) = \varphi(W_a x(t) + R_a h(t-1) + b_a),
$$
where $a \in \{z, i, f, o\}$ and $\varphi$ is either sigmoid (for $i, f, o$) or tanh (for $z$). We alter the notation of the figure in that we write $h(t)$ instead of $y(t)$, which lets us use the latter for the output variable as we are used to. The LSTM forward rule is 
$$
c(t) = c(t-1) + z(t) \odot i(t) \\
h(t) = \tanh(c(t)) \odot o(t).
$$
To obtain predictions $\hat y(t)$ we facilitate an output layer
$$
\hat y(t) = \sigma(W_y h(t) + b_y).
$$

## Exercise 2: Forward pass of the gates

Consider the layer 
$$
a = \varphi(W x + R h + b).
$$
The modules $a \in \{z, i, o\}$ are called cell input, input gate, and output gate, respectively. The cell input uses $\varphi = \tanh$ whereas input gate and output gate use $\varphi = \sigma$. Implement the forward pass of the class `Gate` by implementing the methods `__init__` and `forward`. The method `__init__` should initialize the parameters uniformly in $[-0.01, 0.01]$. The method `forward` should implement the forward logic of the module. The activation function $\varphi$ should be exchangeable. 

In [92]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import expit as sigmoid

def tanh_der(x):
    return (1-(np.tanh(x)**2))

def sigm_der(x):
    return (sigmoid(x)*(1-sigmoid(x)))

class Gate(object):

    def __init__(self,I,D,T,af,af_der):
        
        self.a=np.zeros((T+1,I)) #bc there is timestep -1
        
        self.W = np.random.uniform(-.01,.01,size=(I,D))
        self.R = np.random.uniform(-.01,.01,size=(I,I))
        self.b = np.random.uniform(-.01,.01,size=(I))
        
        self.af = af
        self.af_der = af_der

    
    def forward(self, x, h, t): #af-->activation function
        self.a[t] = self.af(self.W@x+self.R@h+self.b) #h[t] because h starts at t=-1


########## YOUR SOLUTION HERE ##########

## Exercise 3: Gradients for the gates

To train a gate module, we need the gradients of the loss with respect to the parameters $W, R, b$. Further, if there are layers below that need training, then we also need the gradients with respect to $x$ and $h$. Given the gradient $\nabla_a L$, derive expressions for the gradients w.r.t $W, R, b, x, h$. 

########## YOUR SOLUTION HERE ##########

From the Forum: "in this exercise we consider the gate as an isolated module without any time indices"

Therefore:

$$
\nabla_{a}L=\frac{\partial L}{\partial a}
$$

$$
\nabla_{R}L=\frac{\partial L}{\partial a}\frac{\partial a}{\partial R}= \frac{\partial L}{\partial a}\frac{\partial a}{\partial \varphi} h
$$
 

$$
\nabla_{W}L=\frac{\partial L}{\partial a}\frac{\partial a}{\partial \varphi}\frac{\partial \varphi}{ \partial W} = \frac{\partial L}{\partial a}\frac{\partial a}{\partial \varphi} x
$$

$$
\nabla_{b}L=\frac{\partial L}{\partial a}\frac{\partial a}{\partial \varphi}\frac{\partial \varphi}{ \partial b} = \frac{\partial L}{\partial a}\frac{\partial a}{\partial \varphi}
$$

$$
\nabla_{x}L=\frac{\partial L}{\partial a}\frac{\partial a}{\partial \varphi}\frac{\partial \varphi}{ \partial x} = \frac{\partial L}{\partial a}\frac{\partial a}{\partial \varphi} W
$$

$$
\nabla_{h}L=\frac{\partial L}{\partial a}\frac{\partial a}{\partial \varphi}\frac{\partial \varphi}{ \partial R} = \frac{\partial L}{\partial a}\frac{\partial a}{\partial \varphi} R
$$

With $\frac{\partial a}{\partial \varphi}$ being either $1-tanh(a)^2$ or $\sigma(a) (1-\sigma(a))$ depending on what $\varphi$ is.

## Exercise 4: Backward pass of the gates

Implement the backward pass of the class `Gate` by implementing the methods `zero_grad`, `backward`, and `update`. The method `zero_grad` should initialize/overwrite the gradients w.r.t. the parameters to zero. The method `backward` should take $\nabla_a L$ as argument and return $\nabla_x L$ and $\nabla_h L$. Moreover, it should add $\nabla_W L, \nabla_R L, \nabla_b L$ to the gradient buffers. The method update should perform a gradient-descent step to update the parameters using a learning rate $\eta$. 

In [93]:
########## YOUR SOLUTION HERE ##########

def zero_grad(self):
    self.nabla_W_L=np.zeros_like(self.W)
    self.nabla_R_L=np.zeros_like(self.R)
    self.nabla_b_L=np.zeros_like(self.b)
    self.nabla_h_L=np.zeros(self.W.shape[0])
    self.nabla_x_L=np.zeros(self.R.shape[0])

def backward(self,nabla_a_L, t, x, h):
    """
    print("nabla_a_L",nabla_a_L.shape)
    print("nabla W L",self.nabla_W_L.shape)
    print("nabla R L",self.nabla_R_L.shape)
    print("self.af_der(self.a[t])",self.af_der(self.a[t]).shape)
    print("h",h.shape)
    print("x",x.shape)
    
    print("outer one",(nabla_a_L@np.diag(self.af_der(self.a[t]))).shape)
    print("outer done",np.outer((nabla_a_L@np.diag(self.af_der(self.a[t]))),x).T.shape)
    """
    self.nabla_W_L+=np.outer((nabla_a_L@np.diag(self.af_der(self.a[t]))),x)
    self.nabla_R_L+=np.outer((nabla_a_L@np.diag(self.af_der(self.a[t]))),h)
    self.nabla_b_L+=nabla_a_L@np.diag(self.af_der(self.a[t]))
    
    nabla_L_x=nabla_a_L@np.diag(self.af_der(self.a[t]))@self.W
    nabla_L_h=nabla_a_L@np.diag(self.af_der(self.a[t]))@self.R

    return nabla_L_x,nabla_L_h

def update(self,eta):
    self.W+=eta*self.nabla_W_L
    self.R+=eta*self.nabla_R_L
    self.b+=eta*self.nabla_b_L

Gate.zero_grad=zero_grad
Gate.backward=backward
Gate.update=update

## Exercise 5: Forward pass of LSTM

Implement the forward pass of the class `LSTM` using the `Gate` module. Again, it should have the methods `__init__` and `forward` using the same initialization scheme as before. The method `forward` should evaluate and return $L(\hat y(T), y(T))$ where $L$ is the binary cross-entropy loss function and $T$ is the index of the last sequence element. Make sure to store all activations which are needed for the backward pass. 

In [94]:
class LSTM(object):
    
    def __init__(self,D,I,K,T):
        
        self.T=T
        self.I=I
        
        self.z = Gate(I,D,T,af=np.tanh,af_der=tanh_der)
        self.i = Gate(I,D,T,af=sigmoid,af_der=sigm_der)
        self.o = Gate(I,D,T,af=sigmoid,af_der=sigm_der)
        self.gates=[self.i,self.o,self.z]
        
        self.h = np.zeros((T+1,I))
        self.c=np.zeros((T+1,I))
        
        self.W=np.random.uniform(-.01,.01,size=(K,I))
        self.R=np.random.uniform(-.01,.01,size=(K,I))
        self.b=np.random.uniform(-.01,.01,size=K)
    
    def loss(self,y,y_hat):
        return -(y*np.log(y_hat)-(1-y)*(np.log(1-y_hat)))
    
    def forward(self, x, y):
        self.x = x
        self.y = y#many to one means only one y
        self.c = np.zeros((T+1,I))
        self.h = np.zeros((T+1,I))
        
        self.y_hat = np.zeros((T,K))
        
        for t in range(self.T):
            self.i.forward(x[t],self.h[t],t)
            self.o.forward(x[t],self.h[t],t)
            self.z.forward(x[t],self.h[t],t)
            
            self.c[t+1] = self.c[t]+self.z.a[t]*self.i.a[t] #𝑐(𝑡−1)[scalar]+𝑧(𝑡)[vector]⊙𝑖(𝑡)[vector] HADAMARD!!
            self.h[t+1] = np.tanh(self.c[t+1])*self.o.a[t] #ℎ(𝑡)[vector]=tanh(𝑐(𝑡))[scalar]⊙𝑜(𝑡)[vector] HADAMARD!!

            self.y_hat[t] = sigmoid(self.W@self.h[t]+self.b) #𝑦̂(𝑡)[vector]=𝜎(𝑊𝑦ℎ(𝑡)[vector]+𝑏𝑦[vector])
        
        return self.loss(y,self.y_hat[-1])


########## YOUR SOLUTION HERE ##########


## Exercise 6: Backward pass of LSTM with BPTT

Realize the backward pass of LSTM by implementing the methods `zero_grad`, `backward`, and `update`. Equations for the gradients can be found in the lecture script section 3.1 "Backpropagation for LSTM". 

In [111]:
########## YOUR SOLUTION HERE ##########

def zero_grad(self):
    self.i.zero_grad()
    self.o.zero_grad()
    self.z.zero_grad()
    

    
    self.grad_L_Ri = np.zeros_like(self.i.R)
    self.grad_L_Ro = np.zeros_like(self.o.R)
    self.grad_L_Rz = np.zeros_like(self.z.R)
    self.grad_L_Wi = np.zeros_like(self.i.W)
    self.grad_L_Wo = np.zeros_like(self.o.W)
    self.grad_L_Wz = np.zeros_like(self.z.W)
    self.grad_L_bi = np.zeros_like(self.i.b)
    self.grad_L_bo = np.zeros_like(self.o.b)
    self.grad_L_bz = np.zeros_like(self.z.b)
    

def backward(self):
       
    """
    grad y_hat is necessary because the derivatives in the script start from h 
    (which in the script is denoted by y)
    --> der L(t)/h(t) = L(t)/yhat(t) yhat(t)/h(t) = L(t)/sigmoid(out(t)) sigmoid(out(t))/out(t) out(t)/h(t)
    
    L/h is zero for all t<T bc many-to-one
    
    """
  
    self.grad_L_h = np.zeros((T,I)) #in the script this is dLdy
    self.grad_L_o = np.zeros((T,I))
    self.grad_L_i = np.zeros((T,I))
    self.grad_L_z = np.zeros((T,I))
    self.grad_L_c = np.zeros(I)
    
    self.delta_i = np.zeros((T,I))
    self.delta_o = np.zeros((T,I))
    self.delta_z = np.zeros((T,I))
    
    self.grad_W = np.zeros_like((self.W))
    self.grad_b = np.zeros_like((self.b))
    
    self.grad_L_h_direct = np.zeros((T,I))
    #grad_L_h_direct --> sigmoidal function is already incorporated in the gradient of the loss
    self.grad_L_h_direct[-1] = (self.y_hat[-1]-self.y)@self.W # D - I x D @ I --> vector of length D; I x D @ D --> vector of length I
    
    grad_i_h = np.zeros((self.I,self.I))
    grad_o_h = np.zeros((self.I,self.I))
    grad_z_h = np.zeros((self.I,self.I))
    
    self.grad_L_h[-1]=self.grad_L_h_direct[-1] # because loss is only at time T and not after T

    
    for t_shift in range(self.T):
        t=self.T-t_shift-1 #minus one because programmers count from zero

        #c[t+1] in the code is c[t] in the formulas because c starts at t = -1
        self.grad_L_o[t] = self.grad_L_h[t]@np.diag(np.tanh(self.c[t+1])) # I x I @ I --> I

        self.grad_L_c = self.grad_L_h[t]@np.diag(self.o.a[t]*tanh_der(self.c[t+1]))+self.grad_L_c
        
        self.grad_L_i[t] = self.grad_L_c@np.diag(self.z.a[t])
        self.grad_L_z[t] = self.grad_L_c@np.diag(self.i.a[t])

                              
        # h has a range of T+1 because there is an h at time t = -1
        # x has range of T 
        grad_Li_x,grad_Li_h = self.i.backward(self.grad_L_i[t], t, self.x[t], self.h[t])
        grad_Lo_x,grad_Lo_h = self.o.backward(self.grad_L_o[t], t, self.x[t], self.h[t])
        grad_Lz_x,grad_Lz_h = self.z.backward(self.grad_L_z[t], t, self.x[t], self.h[t])

        
        self.grad_L_h[t-1] = (  grad_Li_h
                              + grad_Lo_h
                              + grad_Lz_h)

        #print(self.grad_L_h[t])
        #input("grad L h")
        
        #for the deltas h[t] in the code is h[t-1] in the formulas because the array h starts at t=-1
        self.delta_i[t] = self.grad_L_i[t]@np.diag(self.i.af_der(self.i.W@self.x[t]+self.i.R@self.h[t]))
        self.delta_o[t] = self.grad_L_o[t]@np.diag(self.o.af_der(self.o.W@self.x[t]+self.o.R@self.h[t]))
        self.delta_z[t] = self.grad_L_z[t]@np.diag(self.z.af_der(self.z.W@self.x[t]+self.z.R@self.h[t]))

       
        #similar to the case above h[t] in the code is h[t-1] in the formulas because the array h starts at t=-1
        self.grad_L_Ri += np.outer(self.delta_i[t],self.h[t].T)
        self.grad_L_Ro += np.outer(self.delta_o[t],self.h[t].T)
        self.grad_L_Rz += np.outer(self.delta_z[t],self.h[t].T)
    
        self.grad_L_Wi += np.outer(self.delta_i[t],x[t].T)
        self.grad_L_Wo += np.outer(self.delta_o[t],x[t].T)
        self.grad_L_Wz += np.outer(self.delta_z[t],x[t].T)
        
        self.grad_L_bi += self.delta_i[t]
        self.grad_L_bo += self.delta_o[t]
        self.grad_L_bz += self.delta_z[t]
        
    #finally the gradients with respect to the output layer can be calculated by benefit of joining Cross-Entropy-Loss function and the sigmoid
    #same way as in Assignment 1
    self.grad_W += np.outer((self.y - self.y_hat[-1]),self.h[-1])
    self.grad_b += (self.y - self.y_hat[-1])
    
def update(self,eta):
    self.i.update(eta)
    self.o.update(eta)
    self.z.update(eta)

    self.i.R += eta*self.grad_L_Ri
    self.o.R += eta*self.grad_L_Ro
    self.z.R += eta*self.grad_L_Rz
    #print(self.grad_L_Ri,self.grad_L_Ro,self.grad_L_Rz)
    #input()
    
    self.i.W += eta*self.grad_L_Wi
    self.o.W += eta*self.grad_L_Wo
    self.z.W += eta*self.grad_L_Wz
    
    
    #print(self.grad_L_Wi, self.grad_L_Wo, self.grad_L_Wz)
    #input()

    self.i.b += eta*self.grad_L_bi
    self.o.b += eta*self.grad_L_bo
    self.z.b += eta*self.grad_L_bz
    
    #print(self.grad_L_bi, self.grad_L_bo, self.grad_L_bz)
    #input()
    
    self.W += eta*self.grad_W
    self.b += eta*self.grad_b

LSTM.zero_grad = zero_grad
LSTM.backward = backward
LSTM.update = update
    
    

## Exercise 7: LSTM training

Train an LSTM with 32 hidden units on the task from assignment 1 with a sequence length of 100. Tune the number of update steps and the learning rate. After training, evaluate the model on 1000 sequences. 

In [114]:
########## YOUR SOLUTION HERE ##########

D = 1
I = 32
K = 1
T = 100

np.random.seed(0xDEADBEEF)#is it supposed to spell dead beef? moooh!

def generate_data(T):
    
    ########## YOUR SOLUTION HERE ##########
    x_prime=np.random.choice([1,-1],1,p=[.5,.5])
    gaussian=np.random.normal(0,.2,size=(1,(T-1)))
    x = np.insert(gaussian,0,x_prime.T,axis=1)
    y = ((x_prime+1)/2).astype(int)#hacky
    yield [x.T,y]

n_sequences=5

lastima = LSTM(D,I,K,T)
lastima.zero_grad()
data = [generate_data(T).__next__() for _ in range(1000)]
for _ in range(n_sequences):
    for data_point in data:
        x,y=data_point
        loss=lastima.forward(x,y)
        lastima.backward()
    print(loss)
    lastima.update(eta=.1)
    lastima.zero_grad()

print(lastima.i.R)
print(lastima.i.W)
print(lastima.i.b)

print(lastima.o.R)
print(lastima.o.W)
print(lastima.o.b)

print(lastima.z.R)
print(lastima.z.W)
print(lastima.z.b)

print(lastima.W)
print(lastima.b)


    

[0.69037196]
[0.61376081]
[0.44683607]
[0.18511036]
[0.11289835]
[[ 2.00307752e+100  5.46934662e+100 -3.72874378e+100 ... -4.10283661e+100
  -2.97223318e+100 -3.24137659e+100]
 [ 3.77159768e+100  1.03202649e+101 -7.00983455e+100 ... -7.71817062e+100
  -5.61206677e+100 -6.13970059e+100]
 [ 3.89474767e+100  1.06541051e+101 -7.23857462e+100 ... -7.96982434e+100
  -5.79383478e+100 -6.33694122e+100]
 ...
 [ 3.94273956e+100  1.07811136e+101 -7.33168220e+100 ... -8.07072175e+100
  -5.86130882e+100 -6.40558773e+100]
 [ 2.33304593e+100  6.37395046e+100 -4.34015728e+100 ... -4.77670683e+100
  -3.46485429e+100 -3.78248352e+100]
 [ 1.86556021e+100  5.10116258e+100 -3.46727897e+100 ... -3.81737731e+100
  -2.77418916e+100 -3.03314809e+100]]
[[-9.06723439e+101]
 [-1.77927547e+102]
 [-1.59442078e+102]
 [-1.57827277e+102]
 [-2.73162632e+102]
 [-1.20916511e+102]
 [-1.37826349e+102]
 [-1.94640800e+102]
 [-2.59195077e+102]
 [-3.29580667e+101]
 [-1.09741899e+102]
 [-3.05455760e+102]
 [-1.81374579e+102]
 [-

## Exercise 8: The LSTM learning method and RTRL

Analyzing full BPTT for LSTM we find that the recurrent connections through the gates causes most of the intricacy of the backward logic, while the simple inner recurrent connections are responsible for carrying the error signals over long time spans. The *LSTM learning method* truncates the gradient of these outer recurrent connections. In other words, the gates treat $h(t)$ as if they were external inputs and disregard their dependence on the model. This simplifies the gradients and makes RTRL feasible. 

Since $h(t)$ are treated as external inputs, the key to RTRL is the recursion
$$
\frac{\partial c(t)}{\partial \theta} = \sum_{s=1}^t \frac{\partial c(t)}{\partial \theta(s)} = \frac{\partial c(t)}{\partial \theta(t)} + \frac{\partial c(t)}{\partial c(t-1)} \frac{\partial c(t-1)}{\partial \theta},
$$
where $\theta$ is the parameter vector that contains all the model parameters in one large vector. The parameters are shared in time and $\theta(t)$ denotes their usage at time $t$. Above recursion lets us collect the part of the gradient that depends on the past during forward pass. Due to the recurrent weights the size of $\theta$ is $O(I^2)$ and therefore $\frac{\partial c(t)}{\partial \theta}$ is $O(I^3)$. The matrix product on the right-hand side raises the computational complexity of RTRL to $O(I^4)$, which is the reason why RTRL is infeasible for most recurrent architectures. 

Let, e.g., $w_{jk}^i$ denote the element in the $j$-th row and $k$-th column of the matrix $W$ belonging to the input gate $i$. Show that $\frac{\partial c(t)}{\partial \theta}$ for the LSTM learning method has the form 
$$
\frac{\partial c_n(t)}{\partial w_{jk}^i} = z_n(t) i_n(t)(1-i_n(t)) x_k(t) [n=j] \qquad
\frac{\partial c_n(t)}{\partial r_{jk}^i} = z_n(t) i_n(t)(1-i_n(t)) h_k(t-1) [n=j] \qquad
\frac{\partial c_n(t)}{\partial b_j^i} = z_n(t) i_n(t)(1-i_n(t)) [n=j] \\
\frac{\partial c_n(t)}{\partial w_{jk}^z} = i_n(t) (1-z_n(t)^2) x_k(t) [n=j] \qquad
\frac{\partial c_n(t)}{\partial r_{jk}^z} = i_n(t) (1-z_n(t)^2) h_k(t-1) [n=j] \qquad
\frac{\partial c_n(t)}{\partial b_j^z} = i_n(t) (1-z_n(t)^2) [n=j],\\
$$
where $[n=j]$ is the Iverson bracket that evaluates to 1 if the expression inside is true and to 0 otherwise. What is the complexity of RTRL for the LSTM learning method?

########## YOUR SOLUTION HERE ##########

## Exercise 9: Prepare LSTM for RTRL

Write a class `LSTM_RTRL` and implement the methods `__init__`, `zero_grad`, `update`. Do not use the `Gate` class this time but implement the gates directly so they can be trained via RTRL. Make sure to initialize all weights and gradient buffers accordingly. 

In [ ]:
########## YOUR SOLUTION HERE ##########

## Exercise 10: Implement RTRL for the LSTM learning method

Add a method `forward` to the class `LSTM_RTRL` that processes one time step of an input sequence and updates the RTRL gradient buffers using the LSTM learning method. 

In [ ]:
########## YOUR SOLUTION HERE ##########

## Exercise 11: LSTM training with RTRL

Again, train the LSTM on the task from assignment 1 using real-time recurrent learning in combination with the LSTM learning method. Start with a sequence length of 1. What is the maximum sequence length for which the LSTM can learn the task (at reasonable computational cost)? Compare the training behavior to that with BPTT. Explain possible differences.  

In [ ]:
########## YOUR SOLUTION HERE ##########